# Yao.jl
A flexible whitebox Quantum Circuit algorithm design toolkit.

## Feel
We have a register (i.e. a quantum state) $|\psi\rangle$ represented as a vector.

In [52]:
using Yao
reg = register(bit"100")
statevec(reg)

8-element Array{Complex{Float64},1}:
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 1.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im

An predefined operator $X$, represented as a [generalized permute matrix](https://en.wikipedia.org/wiki/Generalized_permutation_matrix).

In [15]:
mat(X)

2×2 Yao.LuxurySparse.PermMatrix{Complex{Float64},Int64}:
    0       1.0+0.0im
 1.0+0.0im     0     

In [20]:
relax!(focus!(reg, [2, 3]), [2, 3]).state
focus!(reg, [2, 3])

LoadError: [91mBoundsError: attempt to access (2, 2, 2)
  at index [6][39m

In [11]:
size(reg.state)

(4, 2)

In [3]:
using BenchmarkTools
using Yao.Intrinsics
nbit = 10
v1 = randn(fill(2,nbit)...)
v1[3] =4
v1[end] = 5
v2 = reshape(v1, 1<<(nbit÷2+1), :)
norder = vcat(collect(nbit÷2+1:nbit),collect(1:nbit÷2));

In [51]:
#relax!(focus!(reg, [2, 3]), [2,3]).state
focus!(reg, [2, 3]).state

4×2 Array{Complex{Float64},2}:
 0.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  0.0+0.0im

In [17]:
using Compat.Test
@test shapeorder((2,2,4,2,2,2,2,5), [2,3,1,4,5,6,8,7]) == ([2, 8, 8, 2, 5], [2, 1, 3, 5, 4])
@test vec(group_permutedims(v1, norder)) == vec(permutedims(v1, norder))

Test Passed


In [18]:
#@benchmark permutedims($v1, $norder)
#@benchmark permutedims($v2, [1,2])
#@benchmark permutedims2($v1, $norder)
#@profile for i = 1:10000 group_permutedims(v1, norder) end
#@code_warntype permutedims2(v1, norder)

In [14]:
reg2 = chain(kron(3, 2=>X), kron(3, 1=>Y)) |> with!(reg)
statevec(reg)

8-element Array{Complex{Float64},1}:
 0.0+0.0im
 0.0+1.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im

In [13]:
statevec(reg2)

8-element Array{Complex{Float64},1}:
 0.0+0.0im
 0.0+0.0im
 1.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im

`chain`'s behavior likes a `Vector`, itself is complete to describe any circuit.

Apply this operator on the 2nd qubit, means calculating $I_2\otimes X\otimes I_2 |\psi\rangle$ 

In [4]:
IXI = kron(3, 2=>X) # size and pair

Total: 3, DataType: Complex{Float64}
kron
└─ 2=>X gate


Notice here, `kron(2, 2=>X)` returns a `KronBlock` instance, `X` is a predefined `XGate` instance, both are `MatrixBlocks`.

`MatrixBlock` implements `mat` interface.

In [5]:
using Yao.Blocks
@assert X isa MatrixBlock
@assert kron(2, 2=>X) isa MatrixBlock
@assert issubtype(MatrixBlock, AbstractBlock)
mat(IXI)

8×8 Yao.LuxurySparse.PermMatrix{Complex{Float64},Int64}:
    0          0       1.0+0.0im  …     0          0          0     
    0          0          0             0          0          0     
 1.0+0.0im     0          0             0          0          0     
    0       1.0+0.0im     0             0          0          0     
    0          0          0             0       1.0+0.0im     0     
    0          0          0       …     0          0       1.0+0.0im
    0          0          0             0          0          0     
    0          0          0          1.0+0.0im     0          0     

In [6]:
reg |> IXI
statevec(reg)

8-element Array{Complex{Float64},1}:
 0.0+0.0im
 0.0+0.0im
 1.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im
 0.0+0.0im

Then we want to build a larger circuit $C = (H\otimes H\otimes H)(I_2\otimes X\otimes I_2)$
![circuit_xhhh.png](attachment:circuit_xhhh.png)

In [7]:
HHH = kron(3, (i=>H for i in 1:3)...)
circuit = chain(IXI, HHH)

Total: 3, DataType: Complex{Float64}
chain
├─ kron
│  └─ 2=>X gate
└─ kron
   ├─ 1=>H gate
   ├─ 2=>H gate
   └─ 3=>H gate


In [37]:
psi = register(bit"000") |> circuit

Default Register (CPU, Complex{Float64}):
    total: 3
    batch: 1
    active: 3

In [54]:
psi |> focus(1,2)

Default Register (CPU, Complex{Float64}):
    total: 3
    batch: 1
    active: 2

In [64]:
using Yao.Intrinsics
res = measure(psi, 2)[1]

1-element Array{Int64,1}:
 3

In [71]:
bitarray(measure(psi, 2, 10)[1], num_bit=2)

2×10 SubArray{Bool,2,BitArray{2},Tuple{UnitRange{Int64},Base.Slice{Base.OneTo{Int64}}},false}:
 false  true  false  false   true  false  false   true  true  true
  true  true  false   true  false  false  false  false  true  true

# Imaging

## Everything is Block
* `Primitive Block` is a block that does not have sub-blocks.
* `Composite Block` is a block that is constructed using sub-blocks.

In [31]:
bigger = kron(10, 2=>IXI, [i=>X for i = 5:6]..., 8=>phase(0.3))

Total: 10, DataType: Complex{Float64}
kron
├─ 2=>kron
│  └─ 2=>X gate
├─ 5=>X gate
├─ 6=>X gate
└─ 8=>Global Phase Gate:0.3


In [30]:
repeat(3, bigger)

Total: 3, DataType: Complex{Float64}
repeat on (1, 2, 3)
└─ kron
   ├─ 2=>kron
   │  └─ 2=>X gate
   ├─ 5=>X gate
   ├─ 6=>X gate
   └─ 8=>Global Phase Gate:0.3


## Luxury Sparse
In our `LuxurySparse` module, we implemented some high performance matrix types

* `PermMatrix`
* `Identity`
* `Diagonal` (extended its `kron` and `multiply` operations)

Making matrix operations more efficient.